In [ ]:
#all_slow

In [ ]:
#hide
from nbdev.showdoc import *

# Tutorial: Sequence Classification Tuning
> A tutorial following the fine-tuning and data API for Sequence Classification

In [ ]:
from adaptnlp.training.core import Strategy, ColReader, RandomSplitter
from adaptnlp.training.sequence_classification import SequenceClassificationTuner, SequenceClassificationDatasets

from fastai.data.transforms import ColSplitter

## Overview
In this tutorial we will cover building a dataset with the AdaptNLP data API for Sequence Classification, and fine-tuning a model towards that dataset. 

Throughout the tutorial you will notice we call from `fastai`, this is because the tuning API is built upon their library 

> Note: Eventually this will change to `fastai_minima`

For our task, we will use the `IMDB_SAMPLE` dataset to help us tune a BERT model

## The Dataset

The first thing we need to do is get some data. We will use fastai's handy `untar_data` function and `URLs` class to download it:

In [ ]:
from fastai.data.external import untar_data, URLs

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE); path.ls()

(#1) [Path('/root/.fastai/data/imdb_sample/texts.csv')]

As you can see, we get a `Path` object showing where the data is stored. Let's open it in `pandas`, as currently the API only supports reading in through `DataFrame`'s:

In [ ]:
import pandas as pd

df = pd.read_csv(path/'texts.csv')

In [ ]:
df.head()

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False
2,negative,"Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt li...",False
3,positive,"Name just says it all. I watched this movie with my dad when it came out and having served in Korea he had great admiration for the man. The disappointing thing about this film is that it only concentrate on a short period of the man's life - interestingly enough the man's entire life would have made such an epic bio-pic that it is staggering to imagine the cost for production.<br /><br />Some posters elude to the flawed characteristics about the man, which are cheap shots. The theme of the movie ""Duty, Honor, Country"" are not just mere words blathered from the lips of a high-brassed offic...",False
4,negative,"This movie succeeds at being one of the most unique movies you've seen. However this comes from the fact that you can't make heads or tails of this mess. It almost seems as a series of challenges set up to determine whether or not you are willing to walk out of the movie and give up the money you just paid. If you don't want to feel slighted you'll sit through this horrible film and develop a real sense of pity for the actors involved, they've all seen better days, but then you realize they actually got paid quite a bit of money to do this and you'll lose pity for them just like you've alr...",False


So we have a `label`, `text`, and an `is_valid` column. Let's see how we can frame this into AdaptNLP

## High Level API

Each `*Tuner` class comes with a factory method to prepare your dataset and generate the `Tuner` class at once. If you want a more exposed API, the next section will introduce you to the mid-level API. Let's see the doc for our function:

In [ ]:
show_doc(SequenceClassificationTuner.from_df)

<h4 id="SequenceClassificationTuner.from_df" class="doc_header"><code>SequenceClassificationTuner.from_df</code><a href="https://github.com/novetta/adaptnlp/tree/master/adaptnlp/training/sequence_classification.py#L139" class="source_link" style="float:right">[source]</a></h4>

> <code>SequenceClassificationTuner.from_df</code>(**`df`**:`DataFrame`, **`text_col`**:`str`=*`'text'`*, **`label_col`**:`str`=*`'labels'`*, **`model_name`**:`str`=*`None`*, **`split_func`**:`callable`=*`_inner`*, **`loss_func`**=*`CrossEntropyLoss()`*, **`metrics`**=*`[<function accuracy at 0x7ff24d17e820>, <fastai.metrics.AccumMetric object at 0x7ff239ec28b0>]`*, **`batch_size`**=*`8`*, **`collate_fn`**=*`default_data_collator`*, **`opt_func`**=*`Adam`*, **`additional_cbs`**=*`None`*, **`expose_fastai_api`**=*`False`*, **`tokenize_func`**:`callable`=*`None`*, **`tokenize_kwargs`**:`dict`=*`{'padding': True}`*, **`auto_kwargs`**:`dict`=*`{}`*, **`lr`**=*`0.001`*, **`splitter`**=*`trainable_params`*, **`cbs`**=*`None`*, **`path`**=*`None`*, **`model_dir`**=*`'models'`*, **`wd`**=*`None`*, **`wd_bn_bias`**=*`False`*, **`train_bn`**=*`True`*, **`moms`**=*`(0.95, 0.85, 0.95)`*)

Convience method to build a [`SequenceClassificationTuner`](/adaptnlptraining.sequence_classification.html#SequenceClassificationTuner) from a Pandas Dataframe



We can see that it wants a `DataFrame`, the name of the text column, the name of the label column, the name of a transformers model, potentially a way to split the data, and some other configurable bits for training (such as loss function, optimizers, etc).

We can also pass in specific kwargs to both the `tokenize()` function (such as `padding`, `truncation`, etc) and the `AutoTokenizer.from_pretrained` constructor (`auto_kwargs`)

The defaults for the high-level API will work out of the box for their task. In this case our loss function is CrossEntropy, we have `accuracy` and `F1Score` as our metrics, and our optimizer is `Adam`. Let's use these defaults and build a `Tuner`:

> Note: `ColSplitter` checks if the `is_valid` column is `True`

We also need to write a tokenization function, which tokenizes our text for us. It should take in a single item for us to grab the text from, a `tokenizer`, and some `tokenize_kwargs`. When in use, the class will pull `tokenizer` and `tokenize_kwargs` from `self.tokenizer` and `self.tokenize_kwargs`

In [ ]:
def tok_func(item, tokenizer, tokenize_kwargs): return tokenizer(item['text'], **tokenize_kwargs)

In our case, we want to tokenize only the column `text`, and then pass in any `tokenize_kwargs` that were passed in `SequenceClassificationTuner.from_df`

In [ ]:
tuner = SequenceClassificationTuner.from_df(
    df,
    text_col='text',
    label_col='label',
    model_name='bert-base-uncased',
    split_func=ColSplitter('is_valid'),
    tokenize_func=tok_func,
)

No value for `max_length` set, automatically adjusting to the size of the model and including truncation
Sequence length set to: 512


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

What this did for us is first tokenize our datasets into a `transformers.Dataset`, then it built our `DataLoaders`, before finally setting up a sequence classification model for us to tune with. 

All that's left now is to train! 

We can use the `lr_find` method to find a good learning rate to use, and the use `tune` to train for a few epochs:

> Note: `lr_find` is fastai's Learning Rate Finder, you can read the documentation on it [here](https://docs.fast.ai/callback.schedule#Learner.lr_find)

In [ ]:
lr = tuner.lr_find()

We can then see the recommended learning rate:

In [ ]:
lr

And now let's use it and tune:

> Note: By default it will use the `OneCyclePolicy`, but `CosineAnnealing` and `SGDR` are also available

In [ ]:
tuner.tune(
    epochs=3,
    lr=float(lr.valley),
    strategy=Strategy.OneCycle
)

Since we now have a trained model, let's try and predict with it! We have access to a `.predict` method, which will hand over inference to the adaptnlp inference API:

> Note: this is mostly aimed at seeing how your model performed, for inference you should still call the direct inference API shown [here](https://novetta.github.io/adaptnlp/sequence_classification)

In [ ]:
review = """This movie is garbage! 
It even had Matt Damon and Michael Keaton in the lead roles, but it was still terrible. 
They couldn't even save it.
"""

In [ ]:
show_doc(SequenceClassificationTuner.predict)

In [ ]:
tuner.predict(review, detail_level='low')

And now you're done, with only three lines of code! Let's try and convert that over to the mid-level API, and remove some of that (heavy) abstraction next.

## Mid-Level API

Let's see now how to recreate that high level API, with the `Datasets` and `Tuner` APIs:

We'll use the `SequenceClassificationDatasets.from_df` helper function:

In [ ]:
help(SequenceClassificationDatasets.from_df)

As we can see, it expects a `DataFrame`, a text column name our text is stored at, a label column name, splits, and a tokenizer model.

First let's look at how to write our splits. 

These should be in the format of a list of indicies, dictating what goes to the training, and what goes to the validation sets such as: `[[0,1,2],[3,4,5]]`.

Since we have an `is_valid` column, we will bring in `fastai`'s `ColSplitter`, which can read in that `is_valid` column and generate some indicies:

In [ ]:
from fastai.data.transforms import ColSplitter

splitter = ColSplitter('is_valid')

In [ ]:
splits = splitter(df); splits

We can see that 800 indicies went to `[0]`, and 200 indicies went to `[1]`. Now that we have some splits, let's build and tokenize our dataset:

In [ ]:
dset = SequenceClassificationDatasets.from_df(
    df = df,
    text_col = 'text',
    label_col = 'label',
    splits = splits,
    tokenizer_name = 'bert-base-uncased',
    tokenize = True,
    tokenize_func = tok_func,
    tokenize_kwargs = {'padding':True}
)

Once we have our tokenized `Dataset`, all we need to do is build our `DataLoaders`:

In [ ]:
dls = dset.dataloaders(batch_size=8, shuffle_train=True)

Let's make sure a batch looks okay:

In [ ]:
next(iter(dls[0]))

Looks great! Next let's move onto our Tuning:

## Tuning a Model

Next we need to tune a model to our dataset! For this we will use the `SequenceClassificationTuner`, which has some good defaults for our task such as metrics and a loss function to use:

In [ ]:
tuner = SequenceClassificationTuner(dls, model_name='bert-base-uncased', tokenizer=dset.tokenizer)

And now we can `tune`! Each `Tuner` class has 6 functions you can call:
* `lr_find`
* `tune`
* `save`
* `load`
* `predict`

> Note: if you want full access to the fastai `Learner`, add `expose_fastai_api=True` to the constructor, and the tuner will expose everything a `Learner` does

In [ ]:
lr = tuner.lr_find()

In [ ]:
lr.valley

And now we can fit with the `OneCycle` policy:

In [ ]:
tuner.tune(
    epochs=3, 
    lr=float(lr.valley),
    strategy=Strategy.OneCycle
)

Finally we can `save` or `load` our model:

In [ ]:
tuner.save('tuned_model')

In [ ]:
tuner.load('tuned_model');

## Using the AdaptNLP Inference API On Our Trained Model

Another option is to use the in-house inference API on that model we just trained. Let's learn how to use it!

First, let's get rid of our `tuner` and clear some memory:

In [ ]:
del tuner
import torch
torch.cuda.empty_cache()
import gc; gc.collect()

Next we'll import `AdaptNLP`'s `EasySequenceClassifier` module:

In [ ]:
from adaptnlp.sequence_classification import EasySequenceClassifier

Build one:

In [ ]:
classifier = EasySequenceClassifier()

And call `tag_text`, passing in our saved model to point at:

In [ ]:
show_doc(classifier.tag_text)

In [ ]:
classifier.tag_text(
    review,
    model_name_or_path='tuned_model'
)